In [8]:
import os
import tensorflow as tf
import pandas as pd
import keras

In [9]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
from sklearn.model_selection import train_test_split

In [9]:
# Define the model architecture
class SSDModel(tf.keras.Model):
    def __init__(self, num_classes):
        super(SSDModel, self).__init__()
        self.num_classes = num_classes

        # Backbone network (e.g. VGG16, maybe resnet50_v2_imagenet?)
        self.backbone = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

        # Feature extractor layers
        self.feature_extractor = keras.Sequential([
            tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D((2, 2))
        ])

        # SSD layers
        self.ssd_layers = []
        for i in range(6):  # 6 default boxes per feature map
            self.ssd_layers.append(SSDLayer(4 + i, num_classes))

        # Prediction layers
        self.prediction_layers = []
        for i in range(6):  # 6 feature maps
            self.prediction_layers.append(PredictionLayer(num_classes))

    def call(self, inputs):
        # Backbone network
        x = self.backbone(inputs)

        # Feature extractor layers
        x = self.feature_extractor(x)

        # SSD layers
        feature_maps = []
        for i, layer in enumerate(self.ssd_layers):
            x, feature_map = layer(x)
            feature_maps.append(feature_map)

        # Prediction layers
        predictions = []
        for i, layer in enumerate(self.prediction_layers):
            predictions.append(layer(feature_maps[i]))

        # Concatenate predictions
        predictions = tf.concat(predictions, axis=1)

        return predictions

class SSDLayer(keras.layers.Layer):
    def __init__(self, num_defaults, num_classes):
        super(SSDLayer, self).__init__()
        self.num_defaults = num_defaults
        self.num_classes = num_classes

    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.num_defaults * (4 + self.num_classes), (3, 3), activation='linear', padding='same')

    def call(self, inputs):
        x = self.conv(inputs)
        x = tf.reshape(x, (-1, x.shape[1], x.shape[2], self.num_defaults, 4 + self.num_classes))
        return x, x[:, :, :, :, :4]

class PredictionLayer(tf.keras.layers.Layer):
    def __init__(self, num_classes):
        super(PredictionLayer, self).__init__()
        self.num_classes = num_classes

    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.num_classes, (3, 3), activation='softmax', padding='same')

    def call(self, inputs):
        x = self.conv(inputs)
        return x

def load_image(path):
    img = tf.io.read_file(path)
    return

def get_label(path):
    file_name = tf.strings.split(path, '/')[-1]
    if'non-vehicle' in file_name:
        return 1
    else:
        return 2

In [3]:
# Import metadata file from the extractor
metadata = pd.read_csv("https://raw.githubusercontent.com/EddieNguyen2012/Vehicle-Detection-Project/main/annotations.csv")
metadata

,Image_Path,Height,Width,Channels,Class
0,data/vehicles/3975.png,64,64,3,1
1,data/vehicles/far (488).png,64,64,3,1
2,data/vehicles/left (730).png,64,64,3,1
3,data/vehicles/far (172).png,64,64,3,1
4,data/vehicles/348.png,64,64,3,1
...,...,...,...,...,...
17755,data/non-vehicles/image878.png,64,64,3,0
17756,data/non-vehicles/image3400.png,64,64,3,0
17757,data/non-vehicles/extra2870.png,64,64,3,0
17758,data/non-vehicles/extra652.png,64,64,3,0


In [6]:
for col in ["Height", "Width", "Channels", "Class"]:
  print(metadata.value_counts(col))

Height
64    17760
Name: count, dtype: int64
Width
64    17760
Name: count, dtype: int64
Channels
3    17760
Name: count, dtype: int64
Class
0    8968
1    8792
Name: count, dtype: int64


All images in the dataset has 64x64 dimensions and 3 color channels. The classes in the dataset are also balanced.

In [ ]:
# Create the model
model = SSDModel(num_classes=2)  # 2 classes: vehicle and non-vehicle

# Compile the model
model.compile(optimizer='adam', loss='ssd_loss', metrics=['accuracy'])

# Load the dataset (replace with your dataset path)
dataset_path = '/content/drive/MyDrive/vehicle_dataset'

# Create a dataset object
dataset = tf.data.Dataset.list_files(dataset_path + '/*/*.png')
dataset = dataset.map(lambda x: (load_image(x), get_label(x)))

# Define the size of the training set
train_size = int(0.8 * dataset.cardinality())

# Split the dataset into training and testing subsets
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

# Train the model
model.fit(train_dataset, epochs=5)